In [1]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import scipy.io as sio
import numpy as np

import skimage.io
import sklearn.metrics
import h5py
import sys
import gc

sys.path.append('../Data_Preprocessing/')
sys.path.append('../Visualization/')
from Preprocess_Utilities import *
from Data_Extractor import *
from Visualization import *

/home/admin-u6142160/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


All data (raw & road)

In [ ]:
path_raw_image = "../Data/090085/090085_20170531.h5"
path_road_mask = "../Data/090085/Road_Data/motor_trunk_pri_sec_tert/motor_trunk_pri_sec_tert.tif"

raw_image = np.array(h5py.File(path_raw_image)['scene'])
road_mask = skimage.io.imread(path_road_mask)

In [ ]:
show_image_against_road(raw_image, road_mask, size=-1, show_raw=False, figsize=(150,150), show_plot=False, 
                        save_path="./Result/presentation/road on img")
plt.close()

In [ ]:
plt.close()
plt.figure(figsize=(150, 150))
plt.imshow(road_mask*255, cmap='gray')
plt.show()
plt.close()

splitted dataset

In [2]:
train_path = '../Data/090085/Road_Data/motor_trunk_pri_sec_tert_uncl_track/posneg_seg_coord_split_thr1_128_16_train'
train_h5f = h5py.File(train_path, 'r')
train_pos_topleft_coord = np.array(train_h5f['positive_example'])
train_neg_topleft_coord = np.array(train_h5f['negative_example'])
train_raw_image = np.array(train_h5f['raw_image'])
train_road_mask = np.array(train_h5f['road_mask'])
train_h5f.close()
gc.collect()

43

In [3]:
Train_Data = FCN_Data_Extractor (train_raw_image, train_road_mask, 128,
                             pos_topleft_coord = train_pos_topleft_coord,
                             neg_topleft_coord = train_neg_topleft_coord,
                             normalization = 'mean')

mu =  [ 115.69131349  161.49723211  341.19094609  334.01943174 2424.52057387
 1328.10566523  683.31856912]
class weight [neg= 0.014629, pos= 0.985371]


In [4]:
mask = np.zeros(train_road_mask.shape)
for coord in Train_Data.topleft_coordinate:
    mask[coord[0]:coord[0]+128, coord[1]:coord[1]+128] += 1
mask[np.where(mask>0)] = 0.5

In [5]:
white_train_raw_image = train_raw_image.copy()
white_train_raw_image[np.where(white_train_raw_image==-9999)] = 9999

In [6]:
show_pred_prob_with_raw(white_train_raw_image, mask, train_road_mask, pred_weight=0.8, figsize=(100,100), 
                                show_plot=False, save_path='train_white.png')

In [7]:
show_pred_prob_with_raw(train_raw_image, mask, train_road_mask, pred_weight=0.8, figsize=(100,100), 
                                show_plot=False, save_path='train.png')